# Hooks Samples

In [1]:
var userHomeDir = System.getProperty("user.home");
var localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";
var langgraph4jVersion = "1.8-SNAPSHOT";

In [2]:
%%bash 
rm -rf \{userHomeDir}/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j

In [3]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
%dependency /add org.slf4j:slf4j-jdk14:2.0.9
%dependency /add org.bsc.langgraph4j:langgraph4j-core:\{langgraph4jVersion}

%dependency /resolve

Repository local url: file:///Users/bsorrentino/.m2/repository/ added.
Adding dependency org.slf4j:slf4j-jdk14:2.0.9
Adding dependency org.bsc.langgraph4j:langgraph4j-core:1.8-SNAPSHOT
Solving dependencies
Resolved artifacts count: 4
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-jdk14/2.0.9/slf4j-jdk14-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-api/2.0.9/slf4j-api-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j/langgraph4j-core/1.8-SNAPSHOT/langgraph4j-core-1.8-SNAPSHOT.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/async/async-generator/4.0.0/async-generator-4.0.0.jar


### Imports

In [20]:
import org.bsc.langgraph4j.*;
import org.slf4j.*;

import org.bsc.langgraph4j.action.AsyncCommandAction;
import org.bsc.langgraph4j.action.AsyncNodeActionWithConfig;
import org.bsc.langgraph4j.hook.NodeHook;
import org.bsc.langgraph4j.hook.EdgeHook;
import org.bsc.langgraph4j.prebuilt.MessagesState;
import org.bsc.langgraph4j.state.AgentState;
import org.bsc.langgraph4j.utils.CollectionsUtils;
import org.bsc.langgraph4j.action.Command;
import org.bsc.langgraph4j.utils.EdgeMappings;

import java.util.Map;
import java.util.concurrent.CompletableFuture;

import static org.bsc.langgraph4j.GraphDefinition.END;
import static org.bsc.langgraph4j.GraphDefinition.START;
import static org.bsc.langgraph4j.action.AsyncNodeActionWithConfig.node_async;
import static org.bsc.langgraph4j.action.AsyncCommandAction.command_async;


try( var file = new java.io.FileInputStream("./logging.properties")) { // INITIALIZE LOGGING
    java.util.logging.LogManager.getLogManager().readConfiguration( file );
}


## Hooks for logging

In [21]:


public record WrapCallLogHook<S extends MessagesState<String>>(Logger log) implements NodeHook.WrapCall<S>, EdgeHook.WrapCall<S> {

    @Override
    public CompletableFuture<Map<String, Object>> applyWrap(String nodeId,
                                                            S state,
                                                            RunnableConfig config,
                                                            AsyncNodeActionWithConfig<S> action) {

        log.info("\nnode start: '{}' with state: {}", nodeId, state);

        return action.apply( state, config ).whenComplete( (result, ex ) -> {

            if( ex != null ) {
                return;
            }

            log.info("\nnode end: '{}' with result: {}", nodeId, result);

        });
    }

    @Override
    public CompletableFuture<Command> applyWrap(String sourceId,
                                                S state,
                                                RunnableConfig config,
                                                AsyncCommandAction<S> action) {
        log.info("\nedge start from: '{}' with state: {}", sourceId, state);

        return action.apply( state, config ).whenComplete( (result, ex ) -> {

            if( ex != null ) {
                return;
            }

            log.info("\nedge end: {}", result);

        });
    }
}


In [22]:
    static class State extends MessagesState<String> {

        public State(Map<String, Object> initData) {
            super(initData);
        }

    }

## Apply Logging Hook

In [26]:
AsyncNodeActionWithConfig<State> simpleAction() {
    return node_async( ( state, config ) -> Map.of( "messages", "%s-%d".formatted(config.nodeId(), System.currentTimeMillis()) ) );
}

AsyncCommandAction<State> simpleEdgeAction() {
    return  command_async( ( state, config ) -> {
                return ( state.messages().size() > 4) ? 
                    new Command(END) :
                    new Command( "node_1" );
            });
}

var log = LoggerFactory.getLogger("LG4J");

var hook = new WrapCallLogHook<State>( log );
var workflow = new StateGraph<>(MessagesState.SCHEMA, State::new)
        .addWrapCallNodeHook( hook )
        .addWrapCallEdgeHook( hook )
        .addNode("node_1", simpleAction() )
        .addNode("node_2", simpleAction() )
        .addNode("node_3", simpleAction() )
        .addNode("node_4", simpleAction() )
        .addEdge(START, "node_1")
        .addEdge("node_1", "node_2")
        .addEdge("node_2", "node_3")
        .addEdge("node_3", "node_4")
        .addConditionalEdges( "node_4", simpleEdgeAction(), 
                EdgeMappings.builder()
                    .to("node_1")
                    .toEND()
                    .build())        
        .compile();

var result = workflow.invoke( GraphInput.noArgs(), RunnableConfig.builder().build());
    
log.info( "Workflow execution result:{}", result.orElseThrow() );

START 

node start: 'node_1' with state: {
	messages=[]
} 

node end: 'node_1' with result: {messages=node_1-1769595531202} 

node start: 'node_2' with state: {
	messages=[
	node_1-1769595531202
	]
} 

node end: 'node_2' with result: {messages=node_2-1769595531204} 

node start: 'node_3' with state: {
	messages=[
	node_1-1769595531202
	node_2-1769595531204
	]
} 

node end: 'node_3' with result: {messages=node_3-1769595531204} 

node start: 'node_4' with state: {
	messages=[
	node_1-1769595531202
	node_2-1769595531204
	node_3-1769595531204
	]
} 

node end: 'node_4' with result: {messages=node_4-1769595531205} 

edge start from: 'node_4' with state: {
	messages=[
	node_1-1769595531202
	node_2-1769595531204
	node_3-1769595531204
	node_4-1769595531205
	]
} 

edge end: goto node 'node_1'
 

node start: 'node_1' with state: {
	messages=[
	node_1-1769595531202
	node_2-1769595531204
	node_3-1769595531204
	node_4-1769595531205
	]
} 

node end: 'node_1' with result: {messages=node_1-176959553120